In [8]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Projects/Perturbed-Equilibrium/JPEC.worktrees/vacuum_julia`


# Fourier Analysis Routines

This notebook demonstrates two core routines: **`fouran!`** (Fourier analysis) and **`foranv!`** (inverse Fourier analysis).  
Both functions operate on matrices representing discretized angular grids and Fourier coefficients.

---

## 1. `fouran!` — Forward Fourier Analysis

This routine computes Fourier-transformed coefficients from an input matrix.

**Mathematical form:**

For each mode index $l_1$ in the range  
$j_{\max} = l_{\max} - l_{\min} + 1$,  

$$
gil[m_{00}+i, \, l_{00}+l_1] 
= \sum_{j=1}^{m_{th}} cs[j, l_1] \cdot gij[i, j], 
\quad i = 1, \dots, m_{th}.
$$

- `gij` : input matrix in real space  
- `cs`  : Fourier basis coefficients  
- `gil` : output Fourier coefficient matrix  
- $(m_{00}, l_{00})$ : index offsets  
- $m_{th}$ : number of angular grid points  

---

## 2. `foranv!` — Inverse Fourier Analysis

This routine reconstructs a real-space matrix from the Fourier coefficients.

**Mathematical form:**

For each pair of mode indices $(l_1, l_2)$ in  
$j_{\max} = l_{\max} - l_{\min} + 1$,  

$$
gll[l_2, l_1] 
= \sum_{i=1}^{m_{th}} 
d\theta \cdot 2\pi \cdot cs[i, l_2] \cdot gil[m_{00}+i, l_{00}+l_1],
$$

where  

- $d\theta = \dfrac{2\pi}{m_{th}}$ is the angular step size,  
- `gll` : reconstructed matrix in real space  
- `gil` : Fourier coefficients from `fouran!`  
- `cs`  : Fourier basis coefficients  

---

## Workflow Summary

1. Start with $gij$ (real-space values).  
2. Call **`fouran!`** → produces `gil` (Fourier coefficients).  
3. Call **`foranv!`** → reconstructs `gll` (approximation of original $gij$).  



In [9]:
function fouran!(
    gij::Matrix{Float64},
    gil::Matrix{Float64},
    cs::Matrix{Float64},
    m00::Int,
    l00::Int,
    lmin::Vector{Int},
    lmax::Vector{Int},
    mth::Int
)

    # -------------------------------------------------------------------------
    # Purpose:
    #   This routine performs a truncated Fourier transform of gij onto gil
    #   using Fourier coefficients stored in cs.
    #
    # Inputs:
    #   gij(i,j)   : input matrix of size (mth × mth), the "physical-space" data
    #   cs(j,l)    : Fourier coefficient matrix (mth × jmax1)
    #   m00, l00   : integer offsets in the gil matrix
    #   lmin, lmax : define the active range of Fourier mode indices
    #   mth        : number of θ-grid points (dimension of gij along i, j)
    #
    # Output:
    #   gil(i', l') : matrix updated in-place, where i' = m00 + i and l' = l00 + l
    #
    # -------------------------------------------------------------------------

    # Compute jmax1 like Fortran
    jmax1 = lmax[1] - lmin[1] + 1

    # Zero out relevant gil block
    for l1 in 1:jmax1
        for i in 1:mth
            gil[m00 + i, l00 + l1] = 0.0
        end
    end

    # Accumulate with ll offset (critical to match Fortran)
    for l1 in 1:jmax1
        ll = l1 - 1 + lmin[1]
        for j in 1:mth
            for i in 1:mth
                gil[m00 + i, l00 + l1] += cs[j, l1] * gij[i, j]
            end
        end
    end

    return nothing
end

fouran! (generic function with 1 method)

In [ ]:
function foranv!(gil::Matrix{Float64}, gll::Matrix{Float64}, cs::Matrix{Float64},
                 m00::Int, l00::Int, mth::Int, lmin::Int, lmax::Int,
                 dth::Float64)
    
    # -------------------------------------------------------------------------
    # Purpose:
    #   This routine performs the inverse Fourier transform of gil using the
    #   coefficient matrix cs, producing the real-space matrix gll.
    #
    # Notes:
    #   • The summation is over the θ-grid index i.
    #   • The factor (dth ⋅ twopi) accounts for the discretized integration
    #     over poloidal angle θ (with spacing dth and full 2π periodicity).
    #   • gll is symmetric in the sense that its block depends on l1, l2
    #     but the roles of l1 and l2 are not interchangeable here because
    #     cs is indexed by (i, l2) and gil by (i, l1).
    #
    # Inputs:
    #   gil(i', l') : input Fourier-space data, dimensions (nths2 × nfm2)
    #   cs(i, l2)   : Fourier coefficient matrix (mth × jmax1)
    #   m00, l00    : integer offsets into gil
    #   mth         : number of θ-grid points
    #   lmin, lmax  : Fourier mode index bounds
    #   dth         : angular spacing (2π / mth)
    #
    # Output:
    #   gll(l2, l1) : jmax1 × jmax1 real-space block
    #
    # -------------------------------------------------------------------------

    jmax1 = lmax - lmin + 1

    # Zero out gll block
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            gll[l2, l1] = 0.0
        end
    end

    # Main accumulation (note: gll[l2, l1], not gll[l1, l2])
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            for i in 1:mth
                gll[l2, l1] += dth * cs[i, l2] * gil[m00+i, l00+l1] * 2π
            end
        end
    end

    return gll
end


foranv! (generic function with 1 method)

In [11]:
function test_fourier()
    # Parameters
    mtheta = 200
    mthvac = 900
    mpert  = 34

    # After defglo/global_alloc
    nths  = 905
    nths2 = 1810
    nfm   = 34
    nfm2  = 68

    m00 = 0
    l00 = 34

    # Define lmin/lmax like in arrays.f (dummy example: from 1 to nfm)
    lmin = [1]
    lmax = [nfm]

    # Allocate arrays
    gij = zeros(Float64, nths, nths)
    gil = zeros(Float64, nths2, nfm2)
    cs  = zeros(Float64, nths, nfm)
    x   = zeros(Float64, nths)

    # Allocate gll for foranv test
    gll = zeros(Float64, nfm, nfm)

    # Fill x(i) = 6.28 / i
    for i in 1:nths
        x[i] = 6.28 / i
    end

    # Fill gij(i,j) and cs(i,j)
    for i in 1:nths
        for j in 1:nths
            gij[i,j] = sin(x[i] + x[j])
        end
        for j in 1:nfm
            cs[i,j] = (sin(j * x[i]))^2
        end
    end

    # Call your Fourier routine
    fouran!(gij, gil, cs, m00, l00, lmin, lmax, nths)

    println("gil(1:5,1:5) submatrix starting at (109+1,33+1):")
    for i in 1:5
        println([gil[109+i, 33+j] for j in 1:5])
    end

    mth = nths
    dth = 2π / mth
    gll = zeros(Float64, nfm, nfm)

    foranv!(gil, gll, cs, m00, l00, mth, lmin[1], lmax[1], dth)

    println("gll(1:5,1:5) matrix:")
    for i in 1:5
        println([gll[i,j] for j in 1:5])
    end

end

test_fourier()


gil(1:5,1:5) submatrix starting at (109+1,33+1):
[0.0, 5.758492623190696, 8.519982775454418, 10.099787086819953, 11.7302450407185]
[0.0, 5.756099490628109, 8.512536094163833, 10.087290470562392, 11.713017282668279]
[0.0, 5.753747610409246, 8.505220197384235, 10.075014410727995, 11.696094147093449]
[0.0, 5.75143592692491, 8.49803167245637, 10.062953123235024, 11.679467630240492]
[0.0, 5.749163420208127, 8.490967224332074, 10.051101024364383, 11.663130006138648]
gll(1:5,1:5) matrix:
[2.3789313086264325, 5.239419792794563, 7.826355667110261, 10.284593399378773, 12.795304525965333]
[5.239419792794563, 10.009696479612979, 13.986576309804287, 17.813106486393192, 21.751528156905444]
[7.8263556671102545, 13.986576309804267, 18.84118428634243, 23.555201297958067, 28.43403506496631]
[10.284593399378766, 17.81310648639312, 23.555201297958053, 29.161845740032398, 34.9837133827433]
[12.795304525965348, 21.75152815690543, 28.434035064966384, 34.98371338274334, 41.80016994126332]


## ✅ Verification

All test cases **match** with the Fortran version of the code.

**Fortran Reference Output**

```Test 2: Fourier and Inverse Fourier
gil(1:5,1:5) submatrix starting at (109+1,33+1):
0.000000 5.758477 8.519921 10.099648 11.729998
0.000000 5.756084 8.512475 10.087153 11.712772
0.000000 5.753733 8.505160 10.074878 11.695851
0.000000 5.751421 8.497972 10.062818 11.679226
0.000000 5.749149 8.490908 10.050967 11.662890
gll(1:5,1:5) matrix:
2.378815 5.239105 7.825739 10.283557 12.793731
5.239105 10.009040 13.985464 17.811371 21.749000
7.825739 13.985464 18.839502 23.552753 28.430625
10.283557 17.811371 23.552753 29.158460 34.979160
12.793731 21.749000 28.430625 34.979160 41.794207```
